In [ ]:
def extract_article_with_selenium(url: str, wait_time: int = 5) -> dict:
    options = Options()
    options.add_argument("--headless=new")
    options.add_argument("--disable-blink-features=AutomationControlled")
    options.add_argument("--no-sandbox")
    options.add_argument("--disable-dev-shm-usage")
    options.add_argument("window-size=1920,1080")
    options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64)")

    driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)

    try:
        driver.get(url)
        time.sleep(wait_time)

        html = driver.page_source
        soup = BeautifulSoup(html, "html.parser")

        title = soup.title.string if soup.title else None

        article_tags = soup.find_all(["article", "section", "div"], limit=5)
        article_text = ""

        for tag in article_tags:
            paragraphs = tag.find_all("p")
            if len(paragraphs) > 3:
                article_text = "\n".join(p.get_text(strip=True) for p in paragraphs)
                break

        if not article_text:
            article_text = "\n".join(p.get_text(strip=True) for p in soup.find_all("p"))

        return {
            "url": url,
            "title": title,
            "content": article_text  # 🔥 sem limite aqui!
        }

    finally:
        driver.quit()

# 3. Teste com Bloomberg ou InfoMoney
test_url = "https://www.infomoney.com.br/economia/haddad-confirma-aliquota-de-175-sobre-aplicacoes-financeiras-e-aumento-para-jcp/"  # Substitua por URL válida
result = extract_article_with_selenium(test_url)
print("Título:", result["title"])
print("Trecho do artigo:\n", result["content"])


Título: Haddad confirma alíquota de 17,5% sobre aplicações financeiras e aumento para JCP
Trecho do artigo:
 Reuters
10/06/2025 14h11•Atualizado 4 horas atrás
Publicidade
O ministro da Fazenda, Fernando Haddad, confirmou nesta terça-feira que o governo vai propor uma alíquota unificada de 17,5% de Imposto de Renda sobre rendimentos de aplicações financeiras, e disse que também será proposto aumentar tributação de Juros sobre Capital Próprio (JCP) de 15% para 20%.
Em entrevista a jornalistas após reunião com o presidente Luiz Inácio Lula da Silva, o ministro disse que enviou as medidas à Casa Civil nesta terça, e que espera que elas cheguem à mesa do presidente ainda nesta data.
“Não mexe com o dia a dia da população, eu considerei as medidas muito mais estruturais e justas do ponto de vista tributário. É uma agenda que interessa à Fazenda, fazer justiça tributária”, disse.
Haddad falou ao retornar ao ministério do Palácio da Alvorada, explicando que conversou com Lula o mesmo que trato

In [7]:
import urllib.robotparser
from urllib.parse import urlparse
import requests
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from bs4 import BeautifulSoup
import time

def is_allowed_by_robots(url: str, user_agent: str = "*") -> bool:
    """Verifica se a URL é permitida segundo o robots.txt do domínio."""
    parsed_url = urlparse(url)
    base_url = f"{parsed_url.scheme}://{parsed_url.netloc}"
    robots_url = f"{base_url}/robots.txt"

    rp = urllib.robotparser.RobotFileParser()
    try:
        rp.set_url(robots_url)
        rp.read()
        return rp.can_fetch(user_agent, url)
    except Exception as e:
        print(f"Erro ao acessar robots.txt de {base_url}: {e}")
        return False  # Por padrão, nega se falhar

# URL de exemplo
url = "https://www.bloomberg.com/news/articles/2025-06-10/us-and-mexico-near-deal-to-cut-steel-duties-and-cap-imports/"
user_agent = "Mozilla/5.0"

if is_allowed_by_robots(url, user_agent=user_agent):
    # Configura o Selenium
    options = Options()
    options.add_argument("--headless=new")
    options.add_argument("--disable-blink-features=AutomationControlled")
    options.add_argument(f"user-agent={user_agent}")
    
    driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)
    driver.get(url)
    time.sleep(3)
    html = driver.page_source
    driver.quit()

    # Processa com BeautifulSoup
    soup = BeautifulSoup(html, "html.parser")
    text_parts = [p.text for p in soup.find_all("p")]
    full_text = "\n".join(text_parts)

    print(full_text[:1000], "...")
else:
    print("⚠️ Acesso à URL bloqueado por robots.txt")


Connecting decision makers to a dynamic network of information, people and ideas, Bloomberg quickly and accurately delivers business and financial information, news and insight around the world
Americas+1 212 318 2000
EMEA+44 20 7330 7500
Asia Pacific+65 6212 1000
Connecting decision makers to a dynamic network of information, people and ideas, Bloomberg quickly and accurately delivers business and financial information, news and insight around the world
Americas+1 212 318 2000
EMEA+44 20 7330 7500
Asia Pacific+65 6212 1000
Listen
2:53
The US and Mexico are close to a deal that would remove Trump's 50% tariffs on steel imports up to a certain volume, similar to a deal during Trump's first term.
The agreement would allow US buyers to import Mexican steel duty-free as long as they kept total shipments below a level based on historical trade volumes, with a new cap higher than the previous one.
The deal hasn't been finalized, and Trump would need to sign off on it, with Mexican Economy Mi